In [19]:
%%HTML
<style type='text/css'>
    *{
        # background-color:#E3EDCD;
        # background-color:black;
        # color:white;
        
    }
    h1{
        color:#1976d2;
    }
    h2{
        color:#f57c00;
    }
    h3{
        color:#ba37ff;
    }
    h4{
        color:green;
    }
    table{
        border:1px solid black !important;
        border-collapse:collapse !important;
    }
    th{
        background-color:blueviolet !important;
        text-align:center;
        color:white;
    }
    th,td{
        border:0.1px solid black !important;
        transition:0.2s all liner;
        
    }
    td:hover{
        transform:scale(1.1);
        background-color:orange;
        color:blueviolet;
    }
    .raw{
        white-space:pre;
        color:green;
    }
    #imp{
        color:red;
    }
    #ct{
        text-align:center;
    }
</style>

## 3.11 向量化字符串操作

使用 Python 的一个优势就是字符串处理起来比较容易。在此基础上创
建的 Pandas 同样提供了一系列向量化字符串操作（vectorized string
operation），它们都是在处理（清洗）现实工作中的数据时不可或缺的
功能。在这一节中，我们将介绍 Pandas 的字符串操作，学习如何用它
们对一个从网络采集来的杂乱无章的数据集进行局部清理。

### 3.11.1 Pandas字符串操作简介

前面的章节已经介绍过如何用 NumPy 和 Pandas 进行一般的运算操作，
因此我们也能简便快速地对多个数组元素执行同样的操作，例如：

In [2]:
import numpy as np
x = np.array([2, 3, 5, 7, 11, 13])
x * 2

array([ 4,  6, 10, 14, 22, 26])

向量化操作简化了纯数值的数组操作语法——我们不需要再担心数组的
长度或维度，只需要关心需要的操作。然而，由于 NumPy 并没有为字
符串数组提供简单的接口，因此需要通过繁琐的 for 循环来解决问题：

In [3]:
data = ['peter', 'Paul', 'MARY', 'gUIDO']
[s.capitalize() for s in data]

['Peter', 'Paul', 'Mary', 'Guido']

虽然这么做对于某些数据可能是有效的，但是假如数据中出现了缺失
值，那么这样做就会引起异常，例如：


In [4]:
data = ['peter', 'Paul', None, 'MARY', 'gUIDO']
try:
    [s.capitalize() for s in data]
except BaseException as e:
    print(e)

'NoneType' object has no attribute 'capitalize'


Pandas 为包含字符串的 Series 和 Index 对象提供的 str 属性堪称两
全其美的方法，它既可以满足向量化字符串操作的需求，又可以正确地
处理缺失值。例如，我们用前面的数据 data 创建了一个 Pandas 的
Series：


In [5]:
import pandas as pd
names = pd.Series(data)
names

0    peter
1     Paul
2     None
3     MARY
4    gUIDO
dtype: object

现在就可以直接调用转换大写方法 capitalize() 将所有的字符串变成
大写形式，缺失值会被跳过：

In [6]:
names.str.capitalize()

0    Peter
1     Paul
2     None
3     Mary
4    Guido
dtype: object

在 str 属性后面用 Tab 键，可以看到 Pandas 支持的所有向量化字符串
方法。

### 3.11.2 Pandas字符串方法列表

如果你熟悉 Python 的字符串方法的话，就会发现 Pandas 绝大多数的字
符串语法都很直观，甚至可以列成一个表格。在深入论述后面的内容之
前，让我们先从这一步开始。这一节的示例将采用一些人名来演示

In [7]:
monte = pd.Series(['Graham Chapman', 'John Cleese', 'Terry Gilliam',
'Eric Idle', 'Terry Jones', 'Michael Palin'])


01. 与Python字符串方法相似的方法

几乎所有 Python 内置的字符串方法都被复制到 Pandas 的向量化字
符串方法中。下面的表格列举了 Pandas 的 str 方法借鉴 Python 字
符串方法的内容：


```python
len() lower() translate() islower()
ljust() upper() startswith() isupper()
rjust() find() endswith() isnumeric()
center() rfind() isalnum() isdecimal()
zfill() index() isalpha() split()
strip() rindex() isdigit() rsplit()
rstrip() capitalize() isspace() partition()
lstrip() swapcase() istitle() rpartition()
```

需要注意的是，这些方法的返回值不同，例如 lower() 方法返回
一个字符串 Series：

In [8]:
monte.str.lower()

0    graham chapman
1       john cleese
2     terry gilliam
3         eric idle
4       terry jones
5     michael palin
dtype: object

但是有些方法返回数值：

In [9]:
monte.str.len()

0    14
1    11
2    13
3     9
4    11
5    13
dtype: int64

有些方法返回布尔值

In [10]:
monte.str.startswith('T')

0    False
1    False
2     True
3    False
4     True
5    False
dtype: bool

还有些方法返回列表或其他复合值：

In [11]:
monte.str.split()

0    [Graham, Chapman]
1       [John, Cleese]
2     [Terry, Gilliam]
3         [Eric, Idle]
4       [Terry, Jones]
5     [Michael, Palin]
dtype: object

在接下来的内容中，我们将进一步学习这类由列表元素构成的
Series（series-of-lists）对象。

02. 使用正则表达式的方法

还有一些支持正则表达式的方法可以用来处理每个字符串元素。表
3-4 中的内容是 Pandas 向量化字符串方法根据 Python 标准库的 re
模块函数实现的 API。

<table>
    <tr>
        <th>方法</th>
        <th>描述</th>
    </tr>
    <tr>
        <td>match()</td>
        <td>对每个元素调用 re.match()，返回布尔类型值</td>
    </tr>
    <tr>
        <td>extract()</td>
        <td>对每个元素调用 re.match()，返回匹配的字符串组（groups）</td>
    </tr>
    <tr>
        <td>findall()</td>
        <td>对每个元素调用 re.findall()</td>
    </tr>
    <tr>
        <td>replace()</td>
        <td>用正则模式替换字符串</td>
    </tr>
    <tr>
        <td>contains()</td>
        <td>对每个元素调用 re.search()，返回布尔类型值</td>
    </tr>
    <tr>
        <td>count()</td>
        <td>计算符合正则模式的字符串的数量</td>
    </tr>
    <tr>
        <td>split()</td>
        <td>等价于 str.split()，支持正则表达式</td>
    </tr>
    <tr>
        <td>rsplit()</td>
        <td>等价于 str.rsplit()，支持正则表达式</td>
    </tr>
</table>

<p id='ct'>表3-4：Pandas向量化字符串方法与Python标准库的re模块函数的对应关系
</p>

通过这些方法，你就可以实现各种有趣的操作了。例如，可以提取
元素前面的连续字母作为每个人的名字（first name）：


In [12]:
monte.str.extract('([A-Za-z]+)') #多次匹配A-Z,a-z

,0
0,Graham
1,John
2,Terry
3,Eric
4,Terry
5,Michael


我们还能实现更复杂的操作，例如找出所有开头和结尾都是辅音字
母的名字——这可以用正则表达式中的开始符号（^）与结尾符号
（$）来实现：

<p class='raw'>
    简单理解下面的语法,回忆爬虫中学习的正则语法
    [^...] 表示匹配除了字符组中的所有内容
    ^表示匹配开头
    .*为贪婪匹配,尽可能多的匹配结果
    $为匹配结尾
</p>

<p class='raw',id='imp'>
    注意,下面这行代码[^...],表示的是否定的‘非(not)’操作,
    表示找到所有非AEIOU开头且非aeiou结尾的
</p>

In [13]:
monte.str.findall(r'^[^AEIOU].*[^aeiou]$')

0    [Graham Chapman]
1                  []
2     [Terry Gilliam]
3                  []
4       [Terry Jones]
5     [Michael Palin]
dtype: object

能将正则表达式应用到 Series 与 DataFrame 之中的话，就有可
能实现更多的数据分析与清洗方法。

03. 其他字符串方法

还有其他一些方法也可以实现方便的操作（如表 3-5 所示）。


<table>
    <tr>
        <th>方法</th>
        <th>描述</th>
    </tr>
    <tr>
        <td>get()</td>
        <td>获取元素索引位置上的值，索引从 0 开始</td>
    </tr>
    <tr>
        <td>slice()</td>
        <td>对元素进行切片取值</td>
    </tr>
    <tr>
        <td>slice_replace()</td>
        <td>对元素进行切片替换</td>
    </tr>
    <tr>
        <td>cat() </td>
        <td>连接字符串（此功能比较复杂，建议阅读文档）</td>
    </tr>
    <tr>
        <td>repeat() </td>
        <td>重复元素</td>
    </tr>
    <tr>
        <td>normalize()</td>
        <td>将字符串转换为 Unicode 规范形式</td>
    </tr>
    <tr>
        <td>pad()</td>
        <td>在字符串的左边、右边或两边增加空格</td>
    </tr>
    <tr>
        <td>wrap()</td>
        <td>将字符串按照指定的宽度换行</td>
    </tr>
    <tr>
        <td>join() </td>
        <td>用分隔符连接 Series 的每个元素</td>
    </tr>
    <tr>
        <td>get_dummies() </td>
        <td>按照分隔符提取每个元素的 dummy 变量，转换为独热（onehot）编码的 DataFrame</td>
    </tr>
</table>

(1) 向量化字符串的取值与切片操作。这里需要特别指出的
是，get() 与 slice() 操作可以从每个字符串数组中获取向量化
元素。例如，我们可以通过 str.slice(0, 3) 获取每个字符串数
组的前三个字符。通过 Python 的标准取值方法也可以取得同样的
效果，例如 df.str.slice(0, 3) 等价于 df.str[0:3]：

In [20]:
monte.str[0:3]

0    Gra
1    Joh
2    Ter
3    Eri
4    Ter
5    Mic
dtype: object

df.str.get(i) 与 df.str[i] 的按索引取值效果类似。  
get() 与 slice() 操作还可以在 split() 操作之后使用。例如，  
要获取每个姓名的姓（last name），可以结合使用 split() 与  
get()：  

In [23]:
monte.str.split() #split默认按照空格进行字符串的切分,返回的结果是一个列表

0    [Graham, Chapman]
1       [John, Cleese]
2     [Terry, Gilliam]
3         [Eric, Idle]
4       [Terry, Jones]
5     [Michael, Palin]
dtype: object

In [24]:
monte.str.split().str.get(-1) 
#对于切分后的上表进行向量化字符串操作并且利用字符串方法获取索引为-1的字符串

0    Chapman
1     Cleese
2    Gilliam
3       Idle
4      Jones
5      Palin
dtype: object

(2) 指标变量。另一个需要多花点儿时间解释的是 get_dummies()   
方法。当你的数据有一列包含了若干已被编码的指标（coded     
indicator）时，这个方法就能派上用场了。例如，假设有一个包含    
了某种编码信息的数据集，如 A= 出生在美国、B= 出生在英国、    
C= 喜欢奶酪、D= 喜欢午餐肉：    

In [25]:
full_monte = pd.DataFrame({'name': monte,
    'info': ['B|C|D', 'B|D', 'A|C', 'B|D', 'B|C',
    'B|C|D']})
full_monte

,name,info
0,Graham Chapman,B|C|D
1,John Cleese,B|D
2,Terry Gilliam,A|C
3,Eric Idle,B|D
4,Terry Jones,B|C
5,Michael Palin,B|C|D


get_dummies() 方法可以让你快速将这些指标变量分割成一个独
热编码的 DataFrame（每个元素都是 0 或 1）：

<p class='raw'>
    理解下面的代码,
    将full_monte中的['info']这一列进行字符串向量化,
    并且利用字符串方法,将这一列的数据转化为one-hot编码
</p>

In [26]:
full_monte['info'].str.get_dummies('|')

,A,B,C,D
0,0,1,1,1
1,0,1,0,1
2,1,0,1,0
3,0,1,0,1
4,0,1,1,0
5,0,1,1,1


通过 Pandas 自带的这些字符串操作方法，你就可以建立一个功能
无比强大的字符串处理程序来清洗自己的数据了。

虽然本书将不再继续介绍这些方法，但是希望你仔细阅读 Pandas 在线
文档中“Working with Text Data”（http://pandas.pydata.org/pandasdocs/stable/text.html）节，或者阅读 3.14 节的相关资源。

### 3.11.3 案例：食谱数据库

前面介绍的这些向量化字符串操作方法非常适合用来处理现实中那些凌
乱的数据。下面将通过一个从不同网站获取的公开食谱数据库的案例来
进行演示。我们的目标是将这些食谱数据解析为食材列表，这样就可以
根据现有的食材快速找到食谱。


获取数据的脚本可以在 https://github.com/fictivekin/openrecipes 上找到，
那里还有最新版的数据库链接。

截至 2016 年春，这个数据集已经有 30MB 了。可以通过下面的命令下
载并解压数据：